In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.linalg
import sympy as sy
sy.init_printing() 

In [2]:
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
def round_expr(expr, num_digits):
    return expr.xreplace({n : round(n, num_digits) for n in expr.atoms(sy.Number)})

# <font face="gotham" color="purple"> Computation of Determinants</font>

For $2\times 2$ matrix $A$, the algorithm of determinant is
$$
  A=\left[\begin{matrix} a & b\cr c & d \end{matrix}\right] \qquad\text{is equivalent to}\qquad \text{det}  A=ad-bc
$$

Now we experiment with SymPy

In [5]:
a, b, c, d, e, f, g, h, i = sy.symbols('a, b, c, d, e, f, g, h, i', real = True)

With defined symbols, the algorithms of $2\times 2$ and $3\times 3$ determinants are

In [6]:
A = sy.Matrix([[a, b], [c, d]])
A.det()

In [7]:
B = sy.Matrix([[a, b, c], [d, e, f], [g, h, i]])
B.det()

## <font face="gotham" color="purple"> Cofactor Expansion</font>

 The $(i,j)$-**cofactor** of $  A$ is denoted as $C_{ij}$ given by
 $$C_{i j}=(-1)^{i+j} \operatorname{det} A_{i j} = (-1)^{i+j} M_{ij}$$
 where $M_{ij}$ is the **minor determinant** which excludes $i$th row and $j$th column.

Consider a matrix $  A$
$$A=\left[\begin{array}{rrr}
1 & 5 & 0 \\
2 & 4 & -1 \\
0 & -2 & 0
\end{array}\right]$$
Any determinants can be expanded long its arbitrary row or column, we expand the determinant via the first row
$$\begin{aligned}
\operatorname{det} A &=1 \cdot \operatorname{det}\left[\begin{array}{rr}
4 & -1 \\
-2 & 0
\end{array}\right]-5 \cdot \operatorname{det}\left[\begin{array}{cc}
2 & -1 \\
0 & 0
\end{array}\right]+0 \cdot \operatorname{det}\left[\begin{array}{rr}
2 & 4 \\
0 & -2
\end{array}\right] \\
&=1(0-2)-5(0-0)+0(-4-0)=-2
\end{aligned}$$

The scalars, $1$, $-5$ and $0$, in front of each minor determinant are the elements of first row of $  A$.

In general, the expansions across $i$th row or $j$th column are
$$
\operatorname{det} A=a_{i 1} C_{i 1}+a_{i 2} C_{i 2}+\cdots+a_{i n} C_{i n}
$$
$$
\operatorname{det} A=a_{1 j} C_{1 j}+a_{2 j} C_{2 j}+\cdots+a_{n j} C_{n j}
$$

### <font face="gotham" color="purple"> A SymPy Example of Determinant Expansion</font>

Consider the matrix below and perform a cofactor expansion

In [8]:
A = sy.Matrix([[49, 0, 61], [73, 22, 96], [2, 0, 32]]);A

⎡49  0   61⎤
⎢          ⎥
⎢73  22  96⎥
⎢          ⎥
⎣2   0   32⎦

Cofactor expansion with the column which has two zero(s) involves the least computation burden:

$$\operatorname{det} A = a_{12}(-1)^{1+2}C_{12}+a_{22}(-1)^{2+2}C_{22}+a_{32}(-1)^{3+2}C_{32}$$

We can use SymPy function for calculationg minors: ```sy.matrices.matrices.MatrixDeterminant.minor(A, i, 1)```. Also we define a function for cofactor expansion:

In [9]:
def cof_exp(matrix, c): # c is the col that we want to expand
    detA = 0
    for i in range(matrix.shape[1]): # matrix.shape[1] is the total number of columns
        detA += matrix[i, c]*(-1)**(i+c)*sy.matrices.matrices.MatrixDeterminant.minor(matrix, i, c)
    return detA

In [10]:
cof_exp(A,1)

It's easy to verify the expansion algorithm Sympy's determinant evaluation function.

In [11]:
A.det()

Actually you can experiment with any random matrices with multiple zeros, the function below has the parameter ```percent=70``` which means $70\%$ of element are non-zero.

In [12]:
B = sy.randMatrix(r = 7, min=10, max=50, percent=70);B

⎡0   0   0   16  14  48  0 ⎤
⎢                          ⎥
⎢0   45  17  26  15  50  14⎥
⎢                          ⎥
⎢0   31  31  0   0   33  43⎥
⎢                          ⎥
⎢0   0   39  48  0   28  12⎥
⎢                          ⎥
⎢42  41  25  36  50  25  19⎥
⎢                          ⎥
⎢36  42  17  36  0   13  20⎥
⎢                          ⎥
⎣0   11  0   16  27  36  19⎦

Calculate determinants with our user-defined function

In [13]:
cof_exp(B,1)

Then verify the result of using determinant method  ```.det()```. We can see indeed cofactor expansion works!

In [14]:
B.det()

**Minor matrices** can also be extracted by using code ```sy.matrices.matrices.MatrixDeterminant.minor_submatrix()```, for instance, the $M_{23}$ matrix of $  B$ is

In [24]:
sy.matrices.matrices.MatrixDeterminant.minor_submatrix(B, 1, 2) # Pythonic indexing

⎡0   0   16  14  48  0 ⎤
⎢                      ⎥
⎢0   31  0   0   33  43⎥
⎢                      ⎥
⎢0   0   48  0   28  12⎥
⎢                      ⎥
⎢42  41  36  50  25  19⎥
⎢                      ⎥
⎢36  42  36  0   13  20⎥
⎢                      ⎥
⎣0   11  16  27  36  19⎦

**Cofactor matrix** is the matrix contain all cofactors of original matrix, and function ```.cofactor_matrix()``` can do the job.

$$A=\left[\begin{array}{rrr}
C_{11} & C_{12} & C_{13} \\
C_{21} & C_{22} & C_{23} \\
C_{31} & C_{32} & C_{33}
\end{array}\right]=
\left[\begin{array}{rrr}
(-1)^{1+1}M_{11} & (-1)^{1+2}M_{12} & (-1)^{1+3}M_{13} \\
(-1)^{2+1}M_{21} & (-1)^{2+2}M_{22} & (-1)^{2+3}M_{23} \\
(-1)^{3+1}M_{31} & (-1)^{3+2}M_{32} & (-1)^{3+3}M_{33}
\end{array}\right]
$$



In [26]:
sy.matrices.matrices.MatrixDeterminant.cofactor_matrix(A)

⎡ 704   -2144  -44 ⎤
⎢                  ⎥
⎢  0    1446    0  ⎥
⎢                  ⎥
⎣-1342  -251   1078⎦

## <font face="gotham" color="purple"> Triangular Matrix</font>

If $A$ is triangular matrix, cofactor expansion can be applied repetitively, the outcome will be a product of the elements on the principal diagonal. 

$$
\operatorname{det A}_{n\times n} = \prod_{i=1}^n a_{ii}
$$

where $a_{ii}$ is the diagonal element. 

Here is the proof, start with $A$

$$
A=\left[\begin{array}{cccc}
a_{11} & a_{12} & \cdots & a_{1 n} \\
& a_{22} & \cdots & a_{2 n} \\
& & \ddots & \\
& & & a_{n n}
\end{array}\right]
$$
Cofactor expanding on the first column, 
$$
a_{11} \cdot \left[\begin{array}{cccc}
a_{22} & a_{22} & \cdots & a_{2 n} \\
& a_{33} & \cdots & a_{3 n} \\
& & \ddots & \\
& & & a_{n n}
\end{array}\right]
$$
Continue the cofactor expansion
$$
\operatorname{det} A=a_{11} a_{22} \cdot \left[\begin{array}{cccc}
a_{33} & a_{34} & \cdots & a_{3 n} \\
& a_{44} & \cdots & a_{4 n} \\
& & \ddots & \\
& & & a_{n n}
\end{array}\right]
$$
Iterating the expansion, eventually
$$
\text { Det } A=a_{11} \cdots a_{n-2, n-2} \cdot \left[\begin{array}{cc}
a_{n-1, n-1} & a_{n-1, n} \\
a_{n n}
\end{array}\right]=a_{11} \cdots a_{n n}
$$

Now let's verify with a numeric example, generate a random upper triangular matrix.

In [31]:
A = np.round(np.random.rand(5,5)*100)
A_triu = np.triu(A); A_triu

array([[73., 66., 34.,  9., 55.],
       [ 0.,  6., 65., 91., 54.],
       [ 0.,  0., 14., 15., 41.],
       [ 0.,  0.,  0., 85., 35.],
       [ 0.,  0.,  0.,  0., 24.]])

Compute the determinant with ```np.linalg.det```

In [32]:
np.linalg.det(A_triu)

Extract the diagonal by ```np.diag()```, then calculate the product. The same results!

In [33]:
A_diag = np.diag(A_triu)
np.prod(A_diag)

## <font face="gotham" color="purple"> Properties of Determinants</font>

Determinants has a long list of properties, however they are mostly derived facts from cofactor expansion. So no need to memorise any of them.
    
1. Let $A$ be an $n×n$ square matrix. If one row of $A$ is multiplied by $k$ to produce the matrix $B$, then: $\text{det} B = k\, \text{det} A$.
2. Let $A$ be an $n×n$ square matrix. If two rows of $A$ are interchanged to produce a matrix $B$ , then $\text{det}B=-\text{det}A$.
3. Let $A$ be an $n×n$ square matrix. If a multiple of one row of $A$ is added to another row to produce the matrix $B$, then $\text{det}A=\text{det}B$.
4. If $A$ is an $n×n$ matrix, then $\text{det}A^T = \text{det}A$
5. A square matrix $A$ is invertible if and only if $\text{det}A\neq 0$
6. If $A$ and $B$ are $n×n$ matrices, then $\text{det}AB = (\text{det}{A})(\text{det}{B})$
7. If $A$ is an $n×n$ matrix and $k$ is a scalar, then $\text{det}\,kA=k^n\,\text{det}A$.
8. If $A$ is an invertible square matrix, then $\text{det}A^{-1}=\frac{1}{\det A}$.
</font>

All of them are straightforward, the key is to demonstrate them via cofactor expansion. Here is some unrigorous proofs.

<font face="gotham" color="red">Proof of property 6</font>:
$$\begin{aligned}
|A B| &=\left|E_{p} \cdots E_{1} B\right|=\left|E_{p}\right|\left|E_{p-1} \cdots E_{1} B\right|=\cdots \\
&=\left|E_{p}\right| \cdots\left|E_{1}\right||B|=\cdots=\left|E_{p} \cdots E_{1}\right||B| \\
&=|A||B|
\end{aligned}$$

<font face="gotham" color="red">Proof of property 7</font>:

Because $\text{det} B = k\, \text{det} A$, one row of $A$ is multiplied by $k$ to produce $B$.Then multiply all the rows of $A$ by $k$, there will be $n$ $k$'s in front of $\text{det} A$, which is $k^n \text{det} A$

<font face="gotham" color="red">Proof of property 8</font>:
$$\begin{aligned}
&\begin{aligned}
A A^{-1} &=I \\
\left|A A^{-1}\right| &=|I|
\end{aligned}\\
&|A|\left|A^{-1}\right|=1\\
&\left|A^{-1}\right|=\frac{1}{|A|}
\end{aligned}$$

These properties are useful in analytical derivation of some other theorems, however not efficient in numerical computation.

# <font face="gotham" color="purple"> Cramer's Rule</font>

If a linear system has $n$ equations and $n$ variables, an algorithm called **Cramer's Rule** can solve the system in terms of determinants as long as the solution is unique.
$$
A_{n\times n}\mathbf{b}_{n} = \mathbf{x}_n
$$

Some convenient notations are introduced here:

For any $A_{n\times n}$ and vector $\mathbf{b}$, denote $A_i(\mathbf{b})$ as the matrix obtained from replacing the $i$th column of $A$ by $\mathbf{b}$.

$$A_{i}(\mathbf{b})=\left[\begin{array}{lllll}
\mathbf{a}_{1} & \cdots & \mathbf{b} & \cdots & \mathbf{a}_{n}
\end{array}\right]$$

The Cramer's Rule can solve each $x_i$ without solving the whole system
$$x_{i}=\frac{\operatorname{det} A_{i}(\mathbf{b})}{\operatorname{det} A}, \quad i=1,2, \ldots, n$$

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">
<b>Fast Proof of Cramer's Rule</b>：

$$\begin{aligned}
A \cdot I_{i}(\mathbf{x}) &=A\left[\mathbf{e}_{1} \quad \cdots \quad \mathbf{x} \quad \cdots \quad \mathbf{e}_{n}\right]=\left[\begin{array}{llllll}
A \mathbf{e}_{1} & \cdots & A \mathbf{x} & \cdots & A \mathbf{e}_{n}
\end{array}\right] \\
&=\left[\begin{array}{llllll}
\mathbf{a}_{1} & \cdots & \mathbf{b} & \cdots & \mathbf{a}_{n}
\end{array}\right]=A_{i}(\mathbf{b})
\end{aligned}$$
    
where $I_i(\mathbf{x})$ is an identity matrix whose $i$th column replaced by $\mathbf{x}$. With determinant's property,
$$(\operatorname{det} A)\left(\operatorname{det} I_{i}(\mathbf{x})\right)=\operatorname{det} A_{i}(\mathbf{b})$$

$\text{det}I_{i}(\mathbf{x})=x_i$, can be shown by cofactor expansion.
</div> 

## <font face="gotham" color="purple"> A NumPy Example On Cramer's Rule</font>

Consider the system
$$\begin{aligned}
&2 x-y+3 z=-3\\
&3 x+3 y-z=10\\
&-x-y+z=-4
\end{aligned}$$

You have surely known several ways to solve it, but let's test if Cramer's rule works. 

Input the matrices into NumPy arrays.

In [34]:
A = np.array([[2, -1, 3], [3, 3, -1], [-1, -1, 1]])
b = np.array([-3, 10, -4])

In [35]:
A_1b = np.copy(A) # Python variable is a reference tag
A_1b[:,0]=b

A_2b = np.copy(A)
A_2b[:,1]=b

A_3b = np.copy(A)
A_3b[:,2]=b

According to Cramer's rule:

In [36]:
x1 = np.linalg.det(A_1b)/np.linalg.det(A)
x2 = np.linalg.det(A_2b)/np.linalg.det(A)
x3 = np.linalg.det(A_3b)/np.linalg.det(A)
(x1, x2, x3)

We can verify the results by NumPy built-in function ```np.linalg.solve```.

In [37]:
np.linalg.solve(A, b)

array([ 1.,  2., -1.])

Or in a straightforward way $A^{-1}b$

In [38]:
np.linalg.inv(A)@b

array([ 1.,  2., -1.])

All results are the same!

However, remember that Cramer's rule is rarely carried out in practice when solving system of equations, since computational cost (measured by the number of floating operations (flop), are much higher than Gaussian-Jordon elimination.

# <font face="gotham" color="purple"> A Determinant Formula For $A^{-1}$</font>

An alternative algorithm for $A^{-1}$ is
$$A^{-1}=\frac{1}{\operatorname{det} A}\left[\begin{array}{cccc}
C_{11} & C_{21} & \cdots & C_{n 1} \\
C_{12} & C_{22} & \cdots & C_{n 2} \\
\vdots & \vdots & & \vdots \\
C_{1 n} & C_{2 n} & \cdots & C_{n n}
\end{array}\right]$$

where the matrix of cofactors on RHS is the **adjugate** matrix, SymPy function is ```sy.matrices.matrices.MatrixDeterminant.adjugate```. And this is the transpose of the _cofactor matrix_ which we computed using ```sy.matrices.matrices.MatrixDeterminant.cofactor_matrix```

## <font face="gotham" color="purple"> A SymPy Example</font>

Generate a random matrix with $20\%$ of zero elements.

In [46]:
A = sy.randMatrix(5, min=-5, max = 5, percent = 80); A

⎡1   0  0   -4  2 ⎤
⎢                 ⎥
⎢-1  0  0   -2  0 ⎥
⎢                 ⎥
⎢-3  0  1   5   0 ⎥
⎢                 ⎥
⎢3   1  4   0   -1⎥
⎢                 ⎥
⎣1   1  -2  1   -4⎦

Compute the adjugate matrix

In [47]:
A_adjugate = sy.matrices.matrices.MatrixDeterminant.adjugate(A); A_adjugate 

⎡-6   -50  -24  4   -4 ⎤
⎢                      ⎥
⎢221  -39  140  18  106⎥
⎢                      ⎥
⎢-33  35   -8   22  -22⎥
⎢                      ⎥
⎢ 3   -37  12   -2   2 ⎥
⎢                      ⎥
⎣71   -49  36   -6   6 ⎦

We can verify if this really the adjugate of $A$, we pick element of $(1, 3), (2, 4), (5, 4)$ of $A$ to compute the cofactors

In [49]:
(-1)**(1+3) * sy.matrices.matrices.MatrixDeterminant.minor(A, 2, 0) # transpose the index
(-1)**(2+4) * sy.matrices.matrices.MatrixDeterminant.minor(A, 3, 1)
(-1)**(5+4) * sy.matrices.matrices.MatrixDeterminant.minor(A, 3, 4)

Adjugate is the transpose of cofactor matrix, thus we reverse the row and column index when referring to the elements in mind. They are correct as in adjugate matrix, so we have obtained the correct one.

To make results tidy, we need some extra steps.

The ```sy.N()``` is for converting to float approximation, i.e. if you don't like fractions.

In [54]:
A_det = A.det()
A_inv = (1/A_det)*A_adjugate
round_expr(sy.N(A_inv), 4)

⎡-0.0484  -0.4032  -0.1935  0.0323   -0.0323⎤
⎢                                           ⎥
⎢1.7823   -0.3145   1.129   0.1452   0.8548 ⎥
⎢                                           ⎥
⎢-0.2661  0.2823   -0.0645  0.1774   -0.1774⎥
⎢                                           ⎥
⎢0.0242   -0.2984  0.0968   -0.0161  0.0161 ⎥
⎢                                           ⎥
⎣0.5726   -0.3952  0.2903   -0.0484  0.0484 ⎦

Now again, we can verify the results with ```.inv()```

In [55]:
round_expr(sy.N(A.inv()), 4)

⎡-0.0484  -0.4032  -0.1935  0.0323   -0.0323⎤
⎢                                           ⎥
⎢1.7823   -0.3145   1.129   0.1452   0.8548 ⎥
⎢                                           ⎥
⎢-0.2661  0.2823   -0.0645  0.1774   -0.1774⎥
⎢                                           ⎥
⎢0.0242   -0.2984  0.0968   -0.0161  0.0161 ⎥
⎢                                           ⎥
⎣0.5726   -0.3952  0.2903   -0.0484  0.0484 ⎦

Or We can show by difference.

In [44]:
A_inv-A.inv()

⎡0  0  0  0  0⎤
⎢             ⎥
⎢0  0  0  0  0⎥
⎢             ⎥
⎢0  0  0  0  0⎥
⎢             ⎥
⎢0  0  0  0  0⎥
⎢             ⎥
⎣0  0  0  0  0⎦

So Cramer's rule indeed works perfectly.

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">
<b>Short Proof of $A^{-1}$ Formula With Determinants</b><br>
We define $x$ the $j$th column of $A^{-1}$ which satisfies
$$
Ax= e_j
$$

and $e_j$ is the $j$th column of an identity matrix, and $j$th entry of $x$ is the $(i,j)$-entry of $A^{-1}$. By Cramer's rule,

$$\left\{(i, j) \text { -entry of } A^{-1}\right\}=x_{i}=\frac{\operatorname{det} A_{i}\left(\mathbf{e}_{j}\right)}{\operatorname{det} A}$$

The cofactor expansion along column $i$ of $A_i(e_j)$,
$$\operatorname{det} A_{i}\left(\mathbf{e}_{j}\right)=(-1)^{i+j} \operatorname{det} A_{j i}=C_{j i}$$
</div>